# Data

## Download data

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gpiosenka/70-dog-breedsimage-data-set")

print("Path to dataset files:", path)

Using Colab cache for faster access to the '70-dog-breedsimage-data-set' dataset.
Path to dataset files: /kaggle/input/70-dog-breedsimage-data-set


## load data

In [2]:
import pandas as pd
import os

In [19]:
dog_train ='/kaggle/input/70-dog-breedsimage-data-set/train'
dog_valid ='/kaggle/input/70-dog-breedsimage-data-set/valid'

In [20]:
def make_dataframe(data_dir):
    filepaths = []
    labels = []

    for label in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, label)
        if os.path.isdir(class_dir):
            for file in os.listdir(class_dir):
                filepaths.append(os.path.join(class_dir, file))
                labels.append(label)

    return pd.DataFrame({'filename': filepaths, 'class': labels})

In [21]:
train_df = make_dataframe(dog_train)
valid_df = make_dataframe(dog_valid)

In [22]:
train_df.head(400)

,filename,class
0,/kaggle/input/70-dog-breedsimage-data-set/trai...,Cocker
1,/kaggle/input/70-dog-breedsimage-data-set/trai...,Cocker
2,/kaggle/input/70-dog-breedsimage-data-set/trai...,Cocker
3,/kaggle/input/70-dog-breedsimage-data-set/trai...,Cocker
4,/kaggle/input/70-dog-breedsimage-data-set/trai...,Cocker
...,...,...
395,/kaggle/input/70-dog-breedsimage-data-set/trai...,Beagle
396,/kaggle/input/70-dog-breedsimage-data-set/trai...,Beagle
397,/kaggle/input/70-dog-breedsimage-data-set/trai...,Beagle
398,/kaggle/input/70-dog-breedsimage-data-set/trai...,Beagle


## Augmentation data

In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8,1.2],
    fill_mode='nearest'
)

In [24]:
datagen_valid = ImageDataGenerator(rescale=1./255)

In [ ]:
train_gen = datagen_train.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='class',
    target_size=(224,224),
    batch_size = 50,
    class_mode='categorical',
    shuffle=True
    )

In [ ]:
val_gen = datagen_valid.flow_from_dataframe(
    dataframe =valid_df,
    x_col='filename',
    y_col='class',
    target_size=(224,224),
    batch_size = 50,
    class_mode='categorical',
    shuffle=False
)



# Model




## ResNet

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam


# new layers , output layer

In [ ]:
# Load pre-trained EfficientNetB0 without الـ top
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model to save time
for layer in base_model.layers[-4:]:
    layer.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)   
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)               
predictions = Dense(70, activation='softmax')(x)  


model = Model(inputs=base_model.input, outputs=predictions)


## Fine tuning

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [30]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_2         │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_1     │ (None, 224, 224,  │          7 │ rescaling_2[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_3         │ (None, 224, 224,  │          0 │ normalization_1[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_3[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,288,489 (16.36 MB)

 Trainable params: 4,246,466 (16.20 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [ ]:
history = model.fit(train_gen,epochs=5,validation_data=val_gen)

Epoch 1/10
497/497 ━━━━━━━━━━━━━━━━━━━━ 128s 258ms/step - accuracy: 0.7508 - loss: 0.8189 - val_accuracy: 0.0143 - val_loss: 5.2711
Epoch 2/10
497/497 ━━━━━━━━━━━━━━━━━━━━ 126s 254ms/step - accuracy: 0.7666 - loss: 0.7577 - val_accuracy: 0.0129 - val_loss: 7.6200
Epoch 3/10
497/497 ━━━━━━━━━━━━━━━━━━━━ 144s 258ms/step - accuracy: 0.7901 - loss: 0.7034 - val_accuracy: 0.0500 - val_loss: 5.8100
Epoch 4/10
497/497 ━━━━━━━━━━━━━━━━━━━━ 127s 255ms/step - accuracy: 0.7894 - loss: 0.6907 - val_accuracy: 0.0629 - val_loss: 5.3244
Epoch 5/10
497/497 ━━━━━━━━━━━━━━━━━━━━ 126s 254ms/step - accuracy: 0.8000 - loss: 0.6327 - val_accuracy: 0.0143 - val_loss: 6.4916
Epoch 6/10
497/497 ━━━━━━━━━━━━━━━━━━━━ 130s 261ms/step - accuracy: 0.8297 - loss: 0.5783 - val_accuracy: 0.0157 - val_loss: 7.7264
Epoch 7/10
497/497 ━━━━━━━━━━━━━━━━━━━━ 140s 257ms/step - accuracy: 0.8200 - loss: 0.5915 - val_accuracy: 0.0143 - val_loss: 5.7867
Epoch 8/10
161/497 ━━━━━━━━━━━━━━━━━━━━ 1:23 250ms/step - accuracy: 0.8291 -

# Deployment

In [ ]:
model.save('dog_project.keras')


In [ ]:
import tensorflow as tf
temp = tf.keras.models.load_model('dog_project.keras')

In [ ]:
temp.evaluate(val_gen)

In [ ]:
temp.summary()